# Олимпиада НТИ - как выиграть за 30 строк

![](https://pp.userapi.com/c638719/v638719005/2f6da/mFHY83I9u2k.jpg)

### Машинное обучение и анализ данных - задача №1


Описание предметной области. Выявление информации из текста - одна из самых сложных и интересных задач машинного обучения. Речь идет не о разборе предложения или структуре текстов, а о таких вопросах как определения авторства или стиля. Говоря о машинном обучении неизбежно встает вопрос о том, как программе обрабатывать и находить новую информацию в огромных объемах данных, которые даже человек не может осмыслить. Например, нет такого в мире специалиста, который бы мог точно определить авторство любой заметки из 200 авторов начала 20го века. Возможность обучения без учителя, когда вручную отмечаются правильные ответы лишь на небольшом количестве данных - одна из самых актуальных задач машинного обучения. Ведь как только алгоритмы смогут эффективно обучаться на больших объемах без предварительной ручной разметки создание сильного искусственного интеллекта будет сильно ближе.

Описание актуальной задачи. В качестве данных предлагается анализировать личные дневники 19го и начала 20го века. Настоящее авторство текстов - популярный вопрос среди искусствоведов, дискуссии о котором не утихают десятилетиями. Благодаря аналитике больших данных можно не просто сказать к чьему перу относится запись, но и выявить под чей стиль она подходит и выявить важные факты, передав искусствоведам эффективный инструмент анализа. Однако, возможен случай когда у набора записей не осталось сведения об авторах, и список авторов неограничен. Чтобы разобрать эти записи можно применять кластеризацию, и находить схожие заметки. При этом, в обучающей выборке лишь небольшое количество размеченных записей (около 2000), и большое (десятки тысяч) неразмеченных данных. Необходимо реализовать обучение с частичным привлечением учителя (semi-supervised learning).

Всего на сайте около 200 авторов, и 100 000 записей дневников.

В качестве обучающей выборки представлено 100 000 записей, из которых 2.000 будет отмечено авторство.

## Решение

Задача содержит 3 датасета:

Первые два даются участникам, а на основе третьего генерируется score.

Давайте скачаем эти датасеты.

In [ ]:
!wget https://tvorog.me/task1_test.csv

In [ ]:
!wget https://tvorog.me/task1_train.csv

## Датасет с ответами.
Его мы используем только для проверки ответов

In [ ]:
!wget https://tvorog.me/task1_answers.csv

# Посмотрим на данные

[Pandas](http://pandas.pydata.org/) это бог табличной PyData. Неплохие примеры его использования можно найти [тут](http://pandas.pydata.org/pandas-docs/stable/tutorials.html).

In [ ]:
import pandas as pd

In [ ]:
# Pandas позволяет очень просто грузить csv
test = pd.DataFrame.from_csv("task1_test.csv")

In [ ]:
# .head() показывает первые 5 элементов таблицы
test.head()

In [ ]:
train = pd.DataFrame.from_csv("task1_train.csv")

In [ ]:
train.head()

К элементам таблицы можно обращаться двумя способами.

1. train['text']
2. train.text

Они возвращают запрашиваемый столбец. Единственная загвоздка заключается в index колонке. 

train.note_id
```
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-26-ed64215d46cd> in <module>()
----> 1 train.note_id

/home/tvorog/.anaconda3/lib/python3.6/site-packages/pandas/core/generic.py in __getattr__(self, name)
   2742             if name in self._info_axis:
   2743                 return self[name]
-> 2744             return object.__getattribute__(self, name)
   2745 
   2746     def __setattr__(self, name, value):

AttributeError: 'DataFrame' object has no attribute 'note_id'```



In [ ]:
train.index

In [ ]:
# Counter считает количество объектов в итераторе и возвращает tuple (item: count)
from collections import Counter

In [ ]:
# .most_common() сортирует по count
Counter(train.person_id).most_common()

In [ ]:
Counter(test.person_id).most_common()

# Организаторы во всём виноваты

![](https://images5.alphacoders.com/633/633216.jpg)

В задании было сказанно о том, что размеченных авторов примерно 2 тысячи и поэтому нужно использовать semi. Так ли это?

In [ ]:
# В pandas можно выбирать элементы с помощью банальных выражений, это все очень похоже на sql
len(test[test.person_id != -1])

# Baseline

Не смотря на то, что организаторы во всём виноваты, мы с вами хотим выйграть олимпиаду НТИ. Сейчас я покажу на сколько просто это сделать. Наш baseline будет состоять из трёх частей. 

1. Чистка текста
2. Векторизация текста
3. Создание и тренировка модели

Использовать мы будем только test датасет, потому что 40к текстов - это вполне нормальная выборка для обучении модели.

### №1 Чистка данных

Для того, чтобы преобразовать все слова в начальную форму достаточно скачать библиотеку [pymorphy2](https://pymorphy2.readthedocs.io/en/latest/). Она распарсит нужные вам слова и приведет их в начальную форму.

In [ ]:
import pip
pip.main(['install','pymorphy2'])

In [ ]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [ ]:
# Для того, чтобы исключить все лишние знаки будем использовать regexp
import re

In [ ]:
# Уникальных слов, вангую, очень много. Давайте кэшировать.
global_morph = {}

In [ ]:
def normalize(text: str) -> str:
    '''Функция, которая принимает строку и возвращает строку. Чистит текст и сводит к начальной форма'''
    global global_morph
    
    # Приведем все в нижний регистр
    text = text.lower()
    
    # Заменим все бяки *не буквы* на пустоту
    text = re.sub(r'\d+', '', text)
    
    # Возьмём все слова
    words = re.findall(r'\w+', text)
    
    tmp = ""
    
    # Для каждого слова
    for word in words:
        # Если оно закешированно
        if word in global_morph.keys():
            # Возьмём его
            tmp += global_morph[word].normal_form
            tmp += " "
        # Иначе
        else:
            # Закешируем
            global_morph[word] = morph.parse(word)[0]
            
            # Возьмём его
            tmp += global_morph[word].normal_form
            tmp += " "

    return tmp[:-1]

In [ ]:
# Pandas позволяет применить ко всей колонке функцию.
test.text = test.text.apply(normalize)

## №2 Векторизация

Всемогучий [sklearn](http://scikit-learn.org/stable/). Представьте, если бы была библиотека, которая за вас может выйграть олимпиаду... Так вот, она есть. Именуется sklearn. 

![](http://s9.pikabu.ru/post_img/2016/12/12/8/og_og_1481547098267285367.jpg)

Немного wikipedia:

```
TF-IDF (от англ. TF — term frequency, IDF — inverse document frequency) — статистическая мера, используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса. Вес некоторого слова пропорционален количеству употребления этого слова в документе, и обратно пропорционален частоте употребления слова в других документах коллекции.
```

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()

Для начала нужно взять все слова в текстах и посчитать сколько раз они там встречаются *для того, чтобы это было возможно мы и приводили всё в начальную форму*. К счастью это можно сделать в одну строчку.

In [ ]:
vectorizer.fit(test.text)

Дальше разобьём датасет на 2 части. В одной части соберём все записи, которые размечены на авторов. Во вротой части будут записи для которых мы должны предсказать.

In [ ]:
data_train = test[test.person_id != -1]
data_predict =  test[test.person_id == -1]

Для того, чтобы обучить модель нам нужно разбить всё на две части. Возьмём все тексты и положем их в X, возьмём всех авторов и положем их в y. Но поскольку наш `vectorizer` уже знает все слова в текстах, он может нам дать вектор для каждого текста, состоящий из TF-IDF значений. Давайте мы попросим его это сделать.

In [ ]:
X = vectorizer.transform(list(data_train['text']))
y = list(data_train['person_id'])

## №3 Создание и тренировка модели


Поскольку у нас много авторов - это задача регрессии. А если есть задача регрессии - то есть и модель, которая такую задачу решает. И, как не странно, называется она ```LogisticRegression```.

In [ ]:
from sklearn.linear_model import LogisticRegression

В ```LogisticRegression``` на результат влияет только одна константа: ```C```. Для улучшения обобщающей способности получающейся модели, то есть уменьшения эффекта переобучения существует регуляризация. ```C``` - это обратная степень регуляризации. Чем меньще ```C``` тем меньше переобучается модель, но тем сложнее обучить модель. 

In [ ]:
# n_jobs - кол-во процессеров, которое мы используем
model = LogisticRegression(C=170, n_jobs=-1) # создадим модель с C = 170 *белка подсказала*.

# ВАЖНО! Модель со 170 может обучаться очень долго *20 минут*, так что лучше поставить поменьше.

Обучим модель.

In [ ]:
model.fit(X,y)

Теперь наша модель *обучена*. Это такое великолепное состояние, когда она может нам что-то дать. Но для того, чтобы она нам что-то дала, в неё нужно что-то положить!

In [ ]:
X_answer = vectorizer.transform(list(data_predict['text'])) # векторезуем текст
answer = model.predict(X_answer) # и предсказываем по нему

## Done!
#### А что теперь?

Давайте поймём на сколько хороша наша модель.

In [ ]:
# Как я уже писал ранее, все ответы лежат в этом *забавном* csv.
answers = pd.DataFrame.from_csv('task1_answers.csv')

Библиотека, которая за вас решает олимпиаду может вам сказать на сколько вы классные. В ```sklearn``` есть реализация ```accuracy_score```. ```accuracy_score``` - это процент того, сколько раз вы угадали.

In [ ]:
from sklearn.metrics import accuracy_score

#сложно

Дело в том, что нас просили предсказать для каждого текста в ```task1_test.csv``` автора. А это значит, что авторы, которые уже размечены тоже учитываются.

In [ ]:
answers.head()

Давайте сопоставим индексы и ответы

In [ ]:
train_index_to_answer = {x:y for x,y in zip(data_train.index, data_train['person_id'])}

In [ ]:
test_index_to_answer = {x:y for x,y in zip(data_predict.index, answer)}

А теперь сгенерируем полный ответ на задачу.

In [ ]:
full_answer = []

for index in answers.index:
    if index in train_index_to_answer.keys():
        full_answer.append(train_index_to_answer[index])
    else:
        full_answer.append(test_index_to_answer[index])

# Ура. 

Посмотрим на ```score```.

![](http://www.setwalls.ru/pic/201305/1680x1050/setwalls.ru-43884.jpg)

In [ ]:
accuracy_score(answers.person_id, full_answer)

# Что делать?

1. Можно улучшать текущую модель с помощью sklearn.
2. Можно улучшить обработку данных.
3. Можно сделать настоящий ```semi-supervised learning```. Я сделаю отдельную тетрадку для этого. Вы можете помочь мне и сделать Pull Request в [мой репозиторий](https://github.com/xenx/nti-bigdata2017).
4. Можно попробовать свои идеи для этой задачи, например, реализовать [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) или [SGDClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html), а лучше сделать [Pipeline](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) из этого.